In [ ]:
import pandas as pd
import sqlite3


# read csv files
df_Fact = pd.read_csv('/content/BUSINESS-PROCESS.csv')
df_TD = pd.read_csv('/content/timedim.csv')
df_PR = pd.read_csv('/content/phonerate.csv')
df_LO = pd.read_csv('/content/location.csv')


In [ ]:
df_Fact

,ID_time,ID_phoneRate,ID_location_Caller,ID_location_Receiver,Price,NumberOfCalls
0,3,1,877,757,15250,48
1,3,1,878,758,15250,48
2,3,1,879,759,15250,48
3,3,1,880,760,15250,48
4,3,1,881,761,15250,48
...,...,...,...,...,...,...
7803,30,5,910,790,50400,72
7804,30,5,911,791,50400,72
7805,30,5,912,792,50400,72
7806,30,5,913,793,50400,72


In [ ]:
df_TD

,ID_time,DayofWeek,DateMonth,DateYear
0,1,tuesday,Jul-03,2003
1,2,wednesday,Jul-03,2003
2,3,thursday,Jul-03,2003
3,4,friday,Jul-03,2003
4,5,saturday,Jul-03,2003
5,6,friday,Aug-03,2003
6,7,saturday,Aug-03,2003
7,8,sunday,Aug-03,2003
8,9,monday,Aug-03,2003
9,10,tuesday,Aug-03,2003


In [ ]:
df_PR

,ID_phoneRate,phoneRateType
0,1,Notte
1,2,Giorno
2,3,Mattino
3,4,24 ore su 24
4,5,CartaNatale
5,6,Festivi
6,7,Business


In [ ]:
df_LO

,ID_location,City,Province,Region
0,244,Citta439,Provincia30,Regione3
1,245,Citta440,Provincia30,Regione3
2,246,Citta441,Provincia30,Regione3
3,247,Citta442,Provincia30,Regione3
4,248,Citta443,Provincia30,Regione3
...,...,...,...,...
1495,1084,Citta1448,Provincia97,Regione10
1496,1085,Citta1449,Provincia97,Regione10
1497,1086,Citta1450,Provincia97,Regione10
1498,1087,Citta1451,Provincia97,Regione10


In [ ]:
pwd

'/content'

In [ ]:
# connect to database
conn = sqlite3.connect("DMdb")
cur = conn.cursor()


In [ ]:
# load dataframes into database As tables with names BP,DT,PR, LO
df_Fact.to_sql("BP", conn)
df_TD.to_sql("DT",conn)
df_PR.to_sql('PR',conn)
df_LO.to_sql('LO',conn)


1500

In [ ]:
# CHECK IF DATA INSERTED INTO TABLES BP,DT,PR, LO
BP = pd.read_sql('SELECT * FROM BP', conn)
BP

,index,ID_time,ID_phoneRate,ID_location_Caller,ID_location_Receiver,Price,NumberOfCalls
0,0,3,1,877,757,15250,48
1,1,3,1,878,758,15250,48
2,2,3,1,879,759,15250,48
3,3,3,1,880,760,15250,48
4,4,3,1,881,761,15250,48
...,...,...,...,...,...,...,...
7803,7803,30,5,910,790,50400,72
7804,7804,30,5,911,791,50400,72
7805,7805,30,5,912,792,50400,72
7806,7806,30,5,913,793,50400,72


In [ ]:
#EXAMPLE OF GROUP BY QUERY IN SQLITE
M1 = pd.read_sql('''SELECT SUM(Price), dateYear, phoneRateType FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time
INNER JOIN PR ON PR.Id_phoneRate = BP.Id_phoneRate GROUP BY PhoneRateType, DateYear ''', conn)
M1

,SUM(Price),DateYear,phoneRateType
0,29151200,2003,24 ore su 24
1,29151200,2004,24 ore su 24
2,40946000,2003,CartaNatale
3,40946000,2004,CartaNatale
4,33275000,2003,Giorno
5,33275000,2004,Giorno
6,674196000,2003,Mattino
7,674196000,2004,Mattino
8,9768350,2003,Notte
9,9782000,2004,Notte


In [ ]:
#EQUIVALENT OF GROUP BY CUBE (PhoneRateType, DateYear) QUERY IN SQLITE,AS SQLITE OFFERS THE GROUP BY ONLY OPERATOR

M2=pd.read_sql('''SELECT SUM(Price), dateYear, phoneRateType FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time INNER JOIN PR ON PR.Id_phoneRate = BP.Id_phoneRate GROUP BY PhoneRateType, DateYear
UNION ALL SELECT SUM(Price), NULL, phoneRateType FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time INNER JOIN PR ON PR.Id_phoneRate = BP.Id_phoneRate GROUP BY PhoneRateType
UNION ALL SELECT SUM(Price), dateYear, NULL FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time INNER JOIN PR ON PR.Id_phoneRate = BP.Id_phoneRate GROUP BY DateYear
UNION ALL SELECT SUM(Price), NULL, NULL FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time INNER JOIN PR ON PR.Id_phoneRate = BP.Id_phoneRate  ''', conn)


In [ ]:
M2

,SUM(Price),DateYear,phoneRateType
0,29151200,2003.0,24 ore su 24
1,29151200,2004.0,24 ore su 24
2,40946000,2003.0,CartaNatale
3,40946000,2004.0,CartaNatale
4,33275000,2003.0,Giorno
5,33275000,2004.0,Giorno
6,674196000,2003.0,Mattino
7,674196000,2004.0,Mattino
8,9768350,2003.0,Notte
9,9782000,2004.0,Notte


In [ ]:
M3=pd.read_sql('''SELECT DateMonth, DateYear,  SUM(price) as totalIncome, DENSE_RANK() over (ORDER BY SUM(price) DESC) as RankIncome
FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time
 GROUP BY DateMonth, DateYear''',conn)

In [ ]:
M3

,DateMonth,DateYear,totalIncome,RankIncome
0,Aug-03,2003,272836600,1
1,Aug-04,2004,272836600,1
2,Jul-04,2004,272836600,1
3,Jul-03,2003,272822950,2
4,Oct-03,2003,241677000,3
5,Oct-04,2004,241677000,3


In [ ]:
M4 = pd.read_sql ('''SELECT DateMonth, SUM(NumberOfCalls) as TotNumOfCalls, DENSE_RANK() over (ORDER BY SUM(NumberOfCalls) DESC) as RankNumOfCalls FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time AND DateYear=2004
 GROUP BY DateMonth''',conn)

In [ ]:
M4

,DateMonth,TotNumOfCalls,RankNumOfCalls
0,Jul-04,73200,1
1,Aug-04,73200,1
2,Oct-04,64800,2


In [ ]:
cur = conn.cursor()

In [ ]:
# create Materialised View M1 in SQLITE
cur.execute('''CREATE VIEW M1  AS
               SELECT DateMonth, DateYear,  SUM(price) as totalIncome, SUM(NumberOfCalls) as TotNumOfCalls, DENSE_RANK() over (ORDER BY SUM(price) DESC) as RankIncome
               FROM BP INNER JOIN DT ON BP.Id_time = DT.Id_time
              GROUP BY DateMonth, DateYear''')

In [ ]:
m1= pd.read_sql('SELECT * FROM M1', conn)
m1

,DateMonth,DateYear,totalIncome,TotNumOfCalls,RankIncome
0,Aug-03,2003,272836600,73200,1
1,Aug-04,2004,272836600,73200,1
2,Jul-04,2004,272836600,73200,1
3,Jul-03,2003,272822950,73168,2
4,Oct-03,2003,241677000,64800,3
5,Oct-04,2004,241677000,64800,3


In [ ]:
# Rewriting Queries 2,3 using Materialised View M1

M02=pd.read_sql('''SELECT DateMonth, DateYear, SUM(TotNumOfCalls), SUM(totalIncome), DENSE_RANK() over (ORDER BY SUM(totalIncome) DESC) as RankIncome
FROM M1 GROUP BY DateMonth, DateYear''',conn)

In [ ]:
M02

,DateMonth,DateYear,SUM(TotNumOfCalls),SUM(totalIncome),RankIncome
0,Aug-03,2003,73200,272836600,1
1,Aug-04,2004,73200,272836600,1
2,Jul-04,2004,73200,272836600,1
3,Jul-03,2003,73168,272822950,2
4,Oct-03,2003,64800,241677000,3
5,Oct-04,2004,64800,241677000,3


In [ ]:
# Rewriting Queries 2,3 using Materialised View M0
M03=pd.read_sql('''SELECT DateMonth, TotNumOfCalls, totalIncome,  RankIncome
FROM M1  WHERE DateYear=2004''',conn)

In [ ]:
M03

,DateMonth,TotNumOfCalls,totalIncome,RankIncome
0,Aug-04,73200,272836600,1
1,Jul-04,73200,272836600,1
2,Oct-04,64800,241677000,3
